# Environment Setup and Paths
This notebook documents the environment and baseline checks for CNN/VIT image/audio experiments on bird species classification.

- Package manager: UV (pinned deps and kernel)
- Frameworks: PyTorch, torchvision, torchaudio, timm/transformers, librosa, scikit-learn
- Datasets: Xeno-Canto A-M/N-Z v11, CUB-200-2011 v7
- Reproducibility: fixed seeds, deterministic dataloaders, versioned artifacts

# UV environment commands (documentation only, not executed here)
- uv init
- uv add torch torchvision torchaudio timm transformers scikit-learn numpy pandas matplotlib seaborn jupyter ipykernel librosa rich
- uv lock
- python -m ipykernel install --user --name speckitdlbird --display-name "Python (speckitdlbird)"

These commands should be run in a terminal. We record versions and seeds in the next cells.

In [1]:
import os, sys, platform, random
import torch, torchvision, torchaudio
import numpy as np

def set_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # CUDNN determinism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(42)

print("Python:", sys.version)
print("Platform:", platform.platform())
print("CUDA available:", torch.cuda.is_available())
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("Torchaudio:", torchaudio.__version__)

Python: 3.12.11 (main, Jul 11 2025, 22:43:48) [Clang 20.1.4 ]
Platform: Linux-6.16.3-76061603-generic-x86_64-with-glibc2.35
CUDA available: True
Torch: 2.9.1+cu128
Torchvision: 0.24.1+cu128
Torchaudio: 2.9.1+cu128


In [2]:
import importlib
pkgs = ["timm","sklearn","numpy","pandas","librosa","matplotlib","seaborn","jupyter"]
versions = {}
for p in pkgs:
    try:
        m = importlib.import_module(p if p != "sklearn" else "sklearn")
        v = getattr(m, "__version__", "unknown")
    except Exception as e:
        v = f"not-installed ({e})"
    versions[p] = v
print(versions)

{'timm': '1.0.22', 'sklearn': '1.7.2', 'numpy': '2.3.5', 'pandas': '2.3.3', 'librosa': '0.11.0', 'matplotlib': '3.10.7', 'seaborn': '0.13.2', 'jupyter': 'unknown'}


In [3]:
import json
from pathlib import Path

# Save environment info
env_info = {
    'python_version': sys.version,
    'torch': torch.__version__,
    'torchvision': torchvision.__version__,
    'torchaudio': torchaudio.__version__,
    'cuda_available': torch.cuda.is_available(),
}

# Add other package versions
for pkg_name, pkg_import in [('timm', 'timm'), ('sklearn', 'sklearn'), 
                               ('numpy', 'numpy'), ('pandas', 'pandas'),
                               ('librosa', 'librosa'), ('matplotlib', 'matplotlib'), 
                               ('seaborn', 'seaborn')]:
    try:
        m = __import__(pkg_import)
        env_info[pkg_name] = getattr(m, '__version__', 'unknown')
    except ImportError:
        env_info[pkg_name] = 'not installed'

# Save to file
artifacts_dir = Path('../artifacts')
artifacts_dir.mkdir(exist_ok=True)
with open(artifacts_dir / 'env.json', 'w') as f:
    json.dump(env_info, f, indent=2)

print("Environment info saved to artifacts/env.json")
print(json.dumps(env_info, indent=2))

Environment info saved to artifacts/env.json
{
  "python_version": "3.12.11 (main, Jul 11 2025, 22:43:48) [Clang 20.1.4 ]",
  "torch": "2.9.1+cu128",
  "torchvision": "0.24.1+cu128",
  "torchaudio": "2.9.1+cu128",
  "cuda_available": true,
  "timm": "1.0.22",
  "sklearn": "1.7.2",
  "numpy": "2.3.5",
  "pandas": "2.3.3",
  "librosa": "0.11.0",
  "matplotlib": "3.10.7",
  "seaborn": "0.13.2"
}
